In [1]:
#!/usr/local/bin/python3
# solver2021.py : 2021 Sliding tile puzzle solver
#
# Code by: name IU ID
#
# Based on skeleton code by D. Crandall & B551 Staff, September 2021
#

In [2]:
import sys
import numpy as np
from time import time

# Housekeeping functions

In [3]:
ROWS=5
COLS=5

In [4]:
def printable_board(board):
    return [ ('%3d ')*COLS  % board[j:(j+COLS)] for j in range(0, ROWS*COLS, COLS) ]

In [5]:
def parse_board():

    start_state = []
    with open('board1.txt', 'r') as file:
        for line in file:
            start_state += [int(i) for i in line.split()]
        return np.reshape(start_state, (5,5))
            
        

In [6]:
board1 = np.array(parse_board())
board1

array([[12,  6, 10,  1, 23],
       [16, 17, 13,  7, 25],
       [19, 14,  8,  3,  4],
       [21, 15,  9, 20,  5],
       [22, 18, 24,  2, 11]])

In [7]:
goal_state = np.array([x for x in range(1,26)]).reshape(5,5)
goal_state

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [8]:
# check if we've reached the goal
def is_goal(state):
    goal_state = np.array([x for x in range(1,26)]).reshape(5,5)

    return np.all(state==goal_state)

# Successor function

In [10]:
# return a list of possible successor states
def successors(state):
    
    ## outputs a 24 X 5 X 5 numpy array containing all possible board configs from current config##
    
    #left moves
    L1 = np.vstack([np.roll(state[0], -1),state[1:]])
    L2 = np.vstack([state[:1], np.roll(state[1],-1), state[2:]])
    L3 = np.vstack([state[:2], np.roll(state[2],-1), state[3:]])
    L4 = np.vstack([state[:3], np.roll(state[3],-1), state[4:]])
    L5 = np.vstack([state[:4], np.roll(state[4],-1)])

    #right moves
    R1 = np.vstack([np.roll(state[0],1),state[1:]])
    R2 = np.vstack([state[:1], np.roll(state[1],1), state[2:]])
    R3 = np.vstack([state[:2], np.roll(state[2],1), state[3:]])
    R4 = np.vstack([state[:3], np.roll(state[3],1), state[4:]])
    R5 = np.vstack([state[:4], np.roll(state[4],1)])

    #down moves
    D1 = np.vstack([np.roll(state[:,0],1),state[:,1:].T]).T
    D2 = np.vstack([state[:,1].T, np.roll(state[:,1],1), state[:,2:].T]).T
    D3 = np.vstack([state[:,:2].T, np.roll(state[:,2],1), state[:,3:].T]).T
    D4 = np.vstack([state[:,:3].T, np.roll(state[:,3],1), state[:,4:].T]).T
    D5 = np.vstack([state[:,:4].T, np.roll(state[:,4],1), state[:,5:].T]).T

    #up moves
    U1 = np.vstack([np.roll(state[:,0],-1),state[:,1:].T]).T
    U2 = np.vstack([state[:,1].T, np.roll(state[:,1],-1), state[:,2:].T]).T
    U3 = np.vstack([state[:,:2].T, np.roll(state[:,2],-1), state[:,3:].T]).T
    U4 = np.vstack([state[:,:3].T, np.roll(state[:,3],-1), state[:,4:].T]).T
    U5 = np.vstack([state[:,:4].T, np.roll(state[:,4],-1), state[:,5:].T]).T
    
    #ring moves
    flat_board = np.reshape(state, (1,25))

    #outer moves
    oc_ind = np.array([5,0,1,2,3,10,6,7,8,4,15,11,12,13,9,20,16,17,18,14,21,22,23,24,19])
    Oc = np.reshape(flat_board[:,oc_ind],(5,5))

    occ_ind = np.array([1,2,3,4,9,0,6,7,8,14,5,11,12,13,19,10,16,17,18,24,15,20,21,22,23])
    Occ = np.reshape(flat_board[:,occ_ind],(5,5))


    #inner moves
    ic_ind = np.array([0,1,2,3,4,5,11,6,7,9,10,16,12,8,14,15,17,18,13,19,20,21,22,23,24])
    Ic = np.reshape(flat_board[:,ic_ind],(5,5))

    icc_ind = np.array([0,1,2,3,4,5,7,8,13,9,10,6,12,18,14,15,11,16,17,19,20,21,22,23,24])
    Icc = np.reshape(flat_board[:,icc_ind],(5,5))
    
    return np.array([(L1,'L1'),(L2,'L2'),(L3,'L3'),(L4,'L4'),(L5,'L5'),
                     (R1,'R1'),(R2,'R2'),(R3,'R3'),(R4,'R4'),(R5,'R5'),
                     (U1,'U1'),(U2, 'U2'),(U3,'U3'),(U4,'U4'),(U5,'U5'),
                     (D1,'D1'),(D2,'D2'),(D3,'D3'),(D4,'D4'),(D5,'D5'),
                     (Oc,'Oc'),(Occ,'Occ'),(Ic,'Ic'),(Icc,'Icc')], dtype = object)
    

# heuristic functions:

In [11]:
def n_displaced_tiles(state):
    compare = state == goal_state
    return 25 - np.count_nonzero(compare)

In [102]:
def sum_abs_diff(state):
    return np.sum(np.abs(np.subtract(state,goal_state)))

In [122]:
def avg_abs_diff(state):
    abs_diff = np.sum(np.abs(np.subtract(state, goal_state)))
    n_displaced = 25 - np.count_nonzero(state == goal_state)
    return int(abs_diff/n_displaced)



# See if solve function flows correctly

In [12]:
# test to see if flow of solve function works
# easy board is solved in 1 move
easy_board = successors(goal_state)[0][0]
easy_board

array([[ 2,  3,  4,  5,  1],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [13]:
successors(easy_board)[5][0]

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [14]:
np.all(successors(easy_board)[5][0]==goal_state)

True

In [61]:
%time

#initialize fringe
fringe = []
initial_board = easy_board
fringe = [(initial_board,0,'staring point', 0)]# (board, g, 'move', f)

#create list to keep track of items from fringe that are checked
visited = []


while fringe:
    state, g, move, f = fringe.pop(np.argmin([item[3] for item in fringe]))
    visited.append((state, g, move, f))

    if is_goal(state):
        print('goal state found')
        print(state)
        print (f'solve found in {[item[1] for item in visited[1:]]} moves')
        print ([item[2] for item in visited[1:]])
        break
        
    else:
        for board,move in successors(state):
            h = n_displaced_tiles(board)
            f= g+1+h
            fringe.append((board, g+1, move, f))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs
goal state found
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
solve found in [1] moves
['R1']


# A* with Search algorithm 3
no solution found ~ 10 min.  very slow.  I think due to checking against visited list and fringe list for each successor

In [129]:
# solve function a fixed number of iterations

time_0=time()

#initialize fringe
fringe = []
initial_board = board1
fringe = [(initial_board,0,'staring point', 0)]# (board, g, 'move', f)

#create list to keep track of items from fringe have been expanded
visited = []


i = 0
for i in range(100):
    #remove value with lowest f from fringe
    state, g, move, f = fringe.pop(np.argmin([item[3] for item in fringe]))
    
    #add state to closed list
    visited.append((state, g, move, f))
    
    #check if state is goal state
    if is_goal(state):
        print('goal state found')
        print(state)
        print (f'solve found in {[item[1] for item in visited[1:]]} moves')
        print ([item[2] for item in visited[1:]])

    else:
        for board,move in successors(state):
            #calculate and h and f
            h = avg_abs_diff(board)
            f= g+1+h
            
            #check if item is in visited
            if np.any([np.all(board == item) for item, g, move, f in visited]):
                continue
            
            # remove fringe item if successor has lower g
            if np.any([np.all(board== item) for item in [item[0] for item in fringe]]):
                match_idx = [np.all(board == item) for item in [item[0] for item in fringe]].index(1)
                if g < fringe[match_idx][1]:
                    del fringe[match_idx]
                
            # add successor to fringe not already in fringe
            if not np.any([np.all(board == item) for item, g, move, f in fringe]):
                fringe.append((board, g+1, move, f))
i +=1

print(f'100 states checked in {np.round((time()-time_0)/60,2)} mins')

100 states checked in 0.38 mins


In [130]:
print(f'items in fringe: {len(fringe)}')
print(f'items visited: {len(visited)}')
print(f'unique states in visited: {len(np.unique([item[0] for item in visited], axis =0))}')
print(f'f of last item in visited: {visited[-1][3]}')
print(f'lowest f value in visited {min(np.array([item[3] for item in visited][1:]))}')

items in fringe: 2047
items visited: 100
unique states in visited: 100
f of last item in visited: 8
lowest f value in visited 7


In [131]:
# A* with search algorithm 3

# solve function trying to find solution
time_0=time()

#initialize open list for fringe
fringe = []
initial_board = board1
fringe = [(initial_board,0,'staring point', 0)]# (board, g, 'move', f)

#create closed list to keep track of items from fringe have been expanded
visited = []

while fringe:
    #remove state with lowest f from closed list
    state, g, move, f = fringe.pop(np.argmin([item[3] for item in fringe]))
    
    #add state to closed list
    visited.append((state, g, move, f))
    
    #check if state is goal state
    if is_goal(state):
        print('goal state found')
        print(state)
        print (f'solve found in {[item[1] for item in visited[1:]]} moves')
        print ([item[2] for item in visited[1:]])
        break

    else:
        for board,move in successors(state):
            #calculate and h and f
            h = avg_abs_diff(board)
            f= g+1+h
            
            #check if item is in visited
            if np.any([np.all(board == item) for item, g, move, f in visited]):
                continue
            
            # remove fringe item if successor has lower g
            if np.any([np.all(board== item) for item in [item[0] for item in fringe]]):
                match_idx = [np.all(board == item) for item in [item[0] for item in fringe]].index(1)
                if g < fringe[match_idx][1]:
                    del fringe[match_idx]
                
            # add successor to fringe
            fringe.append((board, g+1, move, f))


KeyboardInterrupt: 

In [132]:
#stats after 20 mins
print(len(fringe))
print(len(visited))
print(len(np.unique([item[0] for item in visited], axis =0)))
print(max(np.array([item[3] for item in visited])))
print(min(np.array([item[3] for item in visited][1:])))

17917
934
933
9
7


# search algorithm 2
checks more states, but doesn't get closer to the answer.

In [124]:
# solve function a fixed number of iterations

time_0=time()

#initialize fringe
fringe = []
initial_board = board1
fringe = [(initial_board,0,'staring point', 0)]# (board, g, 'move', f)

#create list to keep track of items from fringe have been expanded
visited = []


i = 0
for i in range(100):
    #remove value with lowest f from fringe
    state, g, move, f = fringe.pop(np.argmin([item[3] for item in fringe]))
    
    #add state to closed list
    visited.append((state, g, move, f))
    
    #check if state is goal state
    if is_goal(state):
        print('goal state found')
        print(state)
        print (f'solve found in {[item[1] for item in visited[1:]]} moves')
        print ([item[2] for item in visited[1:]])

    else:
        for board,move in successors(state):
            #calculate and h and f
            h = avg_abs_diff(board)
            f= g+1+h
            
            fringe.append((board, g+1, move, f))
i +=1

print(f'100 states checked in {np.round((time()-time_0)/60,2)} mins')

100 states checked in 0.0 mins


In [125]:
print(f'items in fringe: {len(fringe)}')
print(f'items visited: {len(visited)}')
print(f'unique states in visited: {len(np.unique([item[0] for item in visited], axis =0))}')
print(f'f of last item in visited: {visited[-1][3]}')
print(f'lowest f value in visited {min(np.array([item[3] for item in visited][1:]))}')

items in fringe: 2301
items visited: 100
unique states in visited: 90
f of last item in visited: 8
lowest f value in visited 7


In [127]:
# A* with search algorithm 2

# solve function trying to find solution
time_0=time()

#initialize open list hold fringe items
fringe = []
initial_board = board1
fringe = [(initial_board,0,'staring point', 0)]# (board, g, 'move', f)

#create closed list to keep track of items from fringe have been expanded
visited = []

while fringe:
    #remove state with lowest f from closed list
    state, g, move, f = fringe.pop(np.argmin([item[3] for item in fringe]))
    
    #add state to closed list
    visited.append((state, g, move, f))
    
    #check if state is goal state
    if is_goal(state):
        print('goal state found')
        print(state)
        print (f'solve found in {[item[1] for item in visited[1:]]} moves')
        print ([item[2] for item in visited[1:]])
        break

    else:
        for board,move in successors(state):
            #calculate and h and f
            h = avg_abs_diff(board)
            f= g+1+h
            
            # add successor to fringe
            fringe.append((board, g+1, move, f))


KeyboardInterrupt: 

In [128]:
print(f'items in fringe: {len(fringe)}')
print(f'items visited: {len(visited)}')
print(f'unique states in visited: {len(np.unique([item[0] for item in visited], axis =0))}')
print(f'f of last item in visited: {visited[-1][3]}')
print(f'lowest f value in visited {min(np.array([item[3] for item in visited][1:]))}')

items in fringe: 610766
items visited: 26555
unique states in visited: 15291
f of last item in visited: 10
lowest f value in visited 7


In [8]:
def solve(initial_board):
    """
    1. This function should return the solution as instructed in assignment, consisting of a list of moves like ["R2","D2","U1"].
    2. Do not add any extra parameters to the solve() function, or it will break our grading and testing code.
       For testing we will call this function with single argument(initial_board) and it should return 
       the solution.
    3. Please do not use any global variables, as it may cause the testing code to fail.
    4. You can assume that all test cases will be solvable.
    5. The current code just returns a dummy solution.
    """
    return ["Oc","L2","Icc", "R4"]

In [320]:
# Please don't modify anything below this line
#
if __name__ == "__main__":
#     if(len(sys.argv) != 2):
#         raise(Exception("Error: expected a board filename"))

    start_state = []
    with open('board1.txt', 'r') as file:#changed sys.argv[1] to run in interactive
        for line in file:
            start_state += [ int(i) for i in line.split() ]
    
    print(start_state)# this was there before

#     if len(start_state) != ROWS*COLS:
#         raise(Exception("Error: couldn't parse start state file"))

    print("Start state: \n" +"\n".join(printable_board(tuple(start_state))))

    print("Solving...")
    route = solve(tuple(start_state))
    
    print("Solution found in " + str(len(route)) + " moves:" + "\n" + " ".join(route))

[12, 6, 10, 1, 23, 16, 17, 13, 7, 25, 19, 14, 8, 3, 4, 21, 15, 9, 20, 5, 22, 18, 24, 2, 11]
Start state: 
 12   6  10   1  23 
 16  17  13   7  25 
 19  14   8   3   4 
 21  15   9  20   5 
 22  18  24   2  11 
Solving...
Solution found in 4 moves:
Oc L2 Icc R4
